## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import numpy as np
import scipy as sp 
import math
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'scipy'

In [ ]:
# Assuming the pulse from the PPG will be restricted between 40 BPM and 240 BPM
#Signal sampling rate in Hz
fs=125
# Let's take small of windows of all the all signals and computing the FFT on just that window.
# 8s time window
window_len=8 
# 2s shit to next window
window_shift=2
# min Bit per minutes
min_bpm=40/60
# max bit per mintes
max_bpm=240/60

In [ ]:
# Compute the fourrier transform
def fourier_transform_computing(sig, fs):
    # np.fft - compute the actual Fourrier Transform coeffcients
    fft=np.abs(np.fft.rfft(sig, len(sig)*2))
    # The frequencies for which we are computing the Fourrier Transform
    freqs=np.fft.rfftfreq(len(sig)*2, 1/fs)
    
    return fft, freqs

In [ ]:
# Bandpass filter helper (using the 40/240 BPM) to create the passd band 

# Always bandpass signals before processing them
# BandPass filter 
# Ref: exo 4
def BandpassFilter(signal, fs):
    ## Filter the signal between min_BPM and max_BPM
    pass_band=(min_bpm, max_bpm)
    # Filtered signal 
    b, a=scipy.butter
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

In [ ]:
!pip uninstall scipy --yes

In [ ]:
# min Bit per minutes
min_bpm=40/60
# max bit per mintes
max_bpm=240/60


def bandpass_filter(signal, fs):
    """filter the signal between 40 and 240 BPM

    Args:
        signal ([np_array]): input signal
        fs ([int]): Hz of input signal

    Returns:
        [np_array]: filtered signal
    """
    pass_band = (min_bpm, max_bpm)
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

In [ ]:
def BandPassFilter(signal, fs):
    pass_band=(min_bpm, max_bpm)
    b, a=scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return scipy.signal.filtfilt(b, a, signal)

In [ ]:
ppg

In [ ]:
ppg = bandpass_filter(ppg, fs)

In [ ]:
import glob
import numpy as np
import scipy as sp
import scipy.io


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]



def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # BandPassFilter the signals before processing them
   

    # Compute pulse rate estimates and estimation confidence.

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, confidence

In [ ]:
Evaluate()

### Visualisation signals with matplotlib

- Plotting your data is a great way to check your assumptions about the data you have.
- Plotting your data is a great way to check your assumptions about the data you have.

In [ ]:
fs=125
sig=sp.io.loadmat(LoadTroikaDataset()[0][0])['sig']
segment=sig[2][100 : 10000]
segment

In [ ]:
ts=np.arange(len(segment))/fs
plt.figure(figsize=(12,8))
plt.plot(ts, segment)
plt.title('Time domain')
plt.xlabel('Time (sec)')

Giving the peaks of the plot it looks like this person jogging ( or in activity)

###### Fourrier Transform

In [ ]:
fs=125
freqs=np.fft.rfftfreq(len(segment), 1/fs)
rfft=np.fft.rfft(segment)
order=np.argsort(np.abs(rfft))[::-1]
most_imp_freqs=list(zip(freqs[order], rfft[order]))

In [ ]:
order

In [ ]:
most_imp_freqs[0]

###### Computing the Fourier Transform (Demonstration)

np.fft module is used to compute the fourrier transform and rfftfreq show the frequencies for which we are computing the fourrier transform. rfft computes the actual Fourrier transforom coeffcient. 

In [ ]:
freqs=np.fft.rfftfreq(len(segment), 1/fs)
fft=np.fft.rfft(segment)

In [ ]:
freqs

In [ ]:
# The actual magnitutde of Fourrier transoform coefficient
np.abs(fft)

In [ ]:
segment

In [ ]:
#The output of the real FFT is half the length of the input signal (plus 1)
len(fft), len(freqs), len(segment)

###### Let's Plot the Fourier transform 

In [ ]:
nfft=len(segment)
fft=np.fft.rfft(segment)
freqs=np.fft.rfftfreq(len(segment), 1/fs)

plt.clf()
plt.figure(figsize=(12,8))
plt.plot(freqs, np.abs(fft))
plt.xticks(np.arange(0, freqs[-1], 2))
plt.title('Frequency domain')
plt.xlabel('Frequency(Hz)')

We do see periodic spikes in the fourier transform, the highest spike is between 2 and 4 Hz

###### The correponding fundamental frequency of fourier transform

In [ ]:
# Computing fundamental frequency and harmonic 
fundamental_frequency = freqs[np.argmax(np.abs(fft[freqs< 2]))]
print(fundamental_frequency)
print('\n')
harmonics_frequency=fundamental_frequency * np.arange(1,9)
print(harmonics_frequency)

###### Visualize the FFT 

In [ ]:
ts=1/fs
plt.figure(figsize=(12,8))
plt.subplot(2,1,1)
plt.plot(ts*100, np.mean(segment))
plt.title('Time-Domain')
plt.xlabel('Time (sec)')
plt.subplot(2,1,2)
plt.plot(freqs, np.abs(fft))
plt.title('Frequency-Domain')
plt.xlabel('Frequency (Hz)')
plt.tight_layout()

#### Bandpass Filter

We can use scipy to bandpass filter our signal for us, A bandpass filter will remove all frequency components outside of a given passband

In [ ]:
# Filter the signal between 40 and 240 BPM
def BandpassFilter(signal, pass_band, fs):
    ## Filter the signal between 40 and 240 BPM
    #pass_band=(40/60, 240/60)
    # Filtered signal 
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

###### Plot the spectogram

In [ ]:
plt.figure(figsize=(12,8))
plt.specgram(sig, Fs=fs, NFFT=250, noverlap=0, xextent=[0, len(sig)/fs])
plt.xlabel('Time(in s)')
plt.ylabel('Frequency(Hz)')

Let's apply the BandPassFilter to see if there difference within the min of BPM=40 to max of BPM=240

In [ ]:
plt.figure(figsize=(12,8))
filtered_sig=BandpassFilter(sig, (40/60, 240/60), fs=fs)
plt.specgram(filtered_sig, Fs=fs, NFFT=250, overlap=125, xextent=[0, len(sig)/fs/60])
plt.xlabel('Time (min)')
plt.ylabel('Frequency (Hz)')

###### PPG SENSOR

In [ ]:
LoadTroikaDataset()

In [ ]:
data_fls, ref_fls = LoadTroikaDataset()

In [ ]:
#signal data
data_fls[0]

In [ ]:
#reference data
ref_fls[0]

###### EDA Signal Processing  of ppg

In [ ]:
# Loads and extracts signals from a troika data file.
data_fls, ref_fls = LoadTroikaDataset()
ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

In [ ]:
#ppg
ppg

In [ ]:
#accx
accx

In [ ]:
#accy
accy

In [ ]:
#accz
accz

###### Visualisation ppg with matplotlib

In [ ]:
fs=125
ts=np.arange(len(ppg))/fs
plt.figure(figsize=(12,8))
plt.plot(ts,ppg)
plt.ylabel('Voltage(mv)')
plt.xlabel('Time(sec)')

###### Let's Plot the Fourier transform of ppg

In [ ]:
nfft=len(ppg)
fft=np.fft.rfft(ppg)
freqs=np.fft.rfftfreq(len(ppg), 1/fs)

plt.clf()
plt.figure(figsize=(12,8))
plt.plot(freqs, np.abs(fft))
plt.xticks(np.arange(0, freqs[-1], 2))
plt.title('Frequency domain')
plt.xlabel('Frequency(Hz)')

In [ ]:
ts=1/fs
plt.figure(figsize=(12,8))
plt.subplot(2,1,1)
plt.plot(ts*100, np.mean(ppg))
plt.title('Time-Domain')
plt.xlabel('Time (sec)')
plt.subplot(2,1,2)
plt.plot(freqs, np.abs(fft))
plt.title('Frequency-Domain')
plt.xlabel('Frequency (Hz)')
plt.tight_layout()

In [ ]:
#PPG peaks - 2-ppg-peaks exercise from Intro to Sensors
ppg_filtered = BandpassFilter(ppg, (min_bpm, max_bpm))
ppg_ft = np.abs(np.fft.rfft(ppg_filtered))
pks= scipy.ppg.find_peaks(ppg_ft)[0]
plt.plot(ppg_filtered)
plt.plot(pks,ppg_filtered[pks], 'r.', ms=5)

###### Spectogram of ppg 

In [ ]:
plt.figure(figsize=(12,8))
plt.specgram(ppg, Fs=fs, NFFT=250, noverlap=125, xextent=[0, len(ppg) / fs / 60])
plt.xlabel("Time (min)")
plt.ylabel("Frequency (Hz)")

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 